# CensorAI project (pet-project)

This project is an AI-model specified for censoring toxic and bad statements in russian telegram chats. I made this project to get more experience in model-building and NLP

In [30]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import tensorflow as tf
import nltk

from nltk.corpus import stopwords
from nltk import SnowballStemmer
from nltk.tokenize import word_tokenize
import string

In [2]:
df = pd.read_csv('./datasets/labeled.csv')

In [3]:
df["toxic"] = df["toxic"].apply(int)

In [4]:
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
2,Собаке - собачья смерть\n,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1


## Let's work now with natural language processing (tokenizing, removing helper words (unions, mezhdometia) etc)

In [5]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\malak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\malak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
snowball_stemmer = SnowballStemmer(language='russian')

helper_words = stopwords.words('russian')

In [7]:
helper_words #

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [8]:
expression = df[df["toxic"] == 1]["comment"].iloc(0)[10]
expression = expression.lower()
expression_tokens = word_tokenize(expression, language='russian')

expression_tokens_without_helpers = [token for token in expression_tokens if (token not in helper_words) and (token not in string.punctuation)]
stemmed_tokens = [snowball_stemmer.stem(i) for i in expression_tokens_without_helpers]
stemmed_tokens

['бактер',
 'теб',
 'поня',
 'ист',
 'е',
 'рик',
 'поподробн',
 'сахар',
 'а.н',
 'веща',
 'какой-т',
 'бред',
 'написа',
 'все',
 'логичн',
 'фраз',
 'обращ',
 'теб',
 'тво',
 'проблем',
 'уч',
 'изъясня',
 'определен',
 'заказа',
 'книг',
 'втор',
 'миров',
 'войн',
 'planeta.ru',
 'о',
 'извин',
 'швал',
 'прост',
 'черн',
 'эт',
 'услов',
 'действительн',
 'заказа',
 'маняграф']

In [9]:
def tokenize_words(expression: str, remove_stopwords: bool = True):
    return [snowball_stemmer.stem(i) for i in [token for token in word_tokenize(expression.lower(), language='russian') if ((token not in helper_words) and remove_stopwords) and (token not in string.punctuation)]]

In [10]:
tokenize_words('Добрый вечер, я диспетчер, выполняю полет даб даб даб')

['добр', 'вечер', 'диспетчер', 'выполня', 'полет', 'даб', 'даб', 'даб']

In [11]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_words(x, remove_stopwords=True))

In [12]:
features = vectorizer.fit_transform(df["comment"])

In [13]:
features

<14412x36127 sparse matrix of type '<class 'numpy.float64'>'
	with 226785 stored elements in Compressed Sparse Row format>

In [14]:
train_df, val_df = train_test_split(df, train_size=0.85, random_state=0)

In [15]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_words(x, remove_stopwords=True))

In [16]:
features = vectorizer.fit_transform(train_df["comment"])

In [17]:
model = RandomForestClassifier(random_state=0)

In [18]:
model.fit(features, train_df["toxic"])

RandomForestClassifier(random_state=0)

In [31]:
pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_words(x, remove_stopwords=True))),
    ("model", LogisticRegression(random_state=0))
])

In [32]:
pipeline.fit(train_df["comment"], train_df["toxic"])

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x000001C626BB4700>)),
                ('model', LogisticRegression(random_state=0))])

In [33]:
pipeline.predict(['Иди нахуй'])

array([1], dtype=int64)

In [34]:
pipeline.predict(['мкирно лучший челик эвер'])

array([0], dtype=int64)

In [23]:
import joblib

In [24]:
accuracy_score(val_df['toxic'], pipeline.predict(val_df['comment']))

0.8283996299722479

In [25]:
import telebot
import time

In [37]:
API_TOKEN = "5941423926:AAHBf1l-NeqRrnd8_ZU-1s0a1txpJFtwW6k"
bot = telebot.TeleBot(API_TOKEN)


@bot.message_handler(content_types=['text'])
def index(message):
    if pipeline.predict([message.text])[0] == 1:
        bot.reply_to(message, 'Не стоит так выражаться!')
        time.sleep(5)
        bot.delete_message(message.chat.id, message.message_id)
bot.infinity_polling()

2022-12-06 17:51:46,694 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2022-12-06 17:51:46,694 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


In [ ]:
df[df["toxic"] == 1]

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
2,Собаке - собачья смерть\n,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1
...,...,...
14404,"ЖЕНЩИНА это ВЕНЕЦ ТВОРЕНИЯ, помните это ваньки...",1
14405,Сейчас все авиакомпании мира вместе со специал...,1
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1


In [36]:
pipeline.predict(['123'])

array([0], dtype=int64)